In [29]:
import pandas as pd

from utils import only_2016_data, clean_out_nan_heavy_rows, group_ages

spatial = pd.read_csv('data/grunnkrets_norway_stripped.csv')
age = pd.read_csv('data/grunnkrets_age_distribution.csv')
income = pd.read_csv('data/grunnkrets_income_households.csv').set_index(['grunnkrets_id', 'year']).add_prefix('income_').reset_index()
households = pd.read_csv('data/grunnkrets_households_num_persons.csv')
submission = pd.read_csv('data/sample_submission.csv')
plaace = pd.read_csv('data/plaace_hierarchy.csv')
busstops = pd.read_csv('data/busstops_norway.csv')

train = pd.read_csv('data/stores_train.csv') #.drop(columns=['year'])
test = pd.read_csv('data/stores_test.csv') 

age_ranges = [
    (0, 19),
    (20, 39),
    (40, 59),
    (60, 79),
    (80, 90),
]

In [30]:
spatial_2016 = only_2016_data(spatial)
income_2016 = only_2016_data(income)
households_2016 = only_2016_data(households)

In [38]:
train_spatial = train.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
muni_avg_revenue = train_spatial.groupby(by='municipality_name', as_index=False)['revenue'].mean()
train_spatial = train_spatial.merge(muni_avg_revenue, on='municipality_name', how='left', suffixes=(None, '_muni_avg'))

train_spatial.head()

,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,revenue,grunnkrets_name,district_name,municipality_name,geometry,area_km2,revenue_muni_avg
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,Hamburger restaurants,6020303,BRAGERNES TORG 13,59.743104,10.204928,MCDONALDS,Magasinet Drammen,17.998,Bragernes sentrum 3,Bragernes sentrum,Drammen,"POLYGON((10.2046156903846 59.7447808519649, 10...",0.155779,7.690191
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,Hamburger restaurants,3010306,NaN,59.913759,10.734031,MCDONALDS,NaN,23.828,Sentrum 3 /rode 6,Sentrum 3,Oslo,"POLYGON((10.7303654475615 59.9107195782207, 10...",0.264278,8.102010
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,Hamburger restaurants,6050102,KONG RINGS GATE 1,60.164751,10.254656,BURGER KING,Kuben Hønefoss,16.099,Sydsiden 2,Hønefoss,Ringerike,"POLYGON((10.2654039198422 60.1639238060368, 10...",0.160152,10.035593
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,Hamburger restaurants,18040102,STORGATA 12,67.283669,14.379796,BURGER KING,Glasshuspassasjen,9.296,Sentrum 1,Sentrum,Bodø,"POLYGON((14.3800126797167 67.2852351710009, 14...",0.095029,9.257164
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,Hamburger restaurants,16017414,NaN,63.358068,10.374832,BURGER KING,Tillertorget,4.528,Tiller-Hårstad 14,Tiller-Hårstad,Trondheim,"POLYGON((10.3709720705149 63.3579302939404, 10...",0.251070,10.343312


In [17]:
def clean_out_nan_heavy_rows(df: pd.DataFrame):
    """Cleans out rows that have no match in the age, spatial, income or household datasets."""

    df2 = df.merge(group_ages(age, age_ranges), on='grunnkrets_id', how='left')
    df2 = df2.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
    df2 = df2.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
    df2 = df2.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

    df_cleaned = df2[
        ~(df2.age_0_19.isna() | df2.couple_children_0_to_5_years.isna() | df2.grunnkrets_name.isna() | df2.income_all_households.isna())
    ]

    print(f'Cleaned out {len(df) - len(df_cleaned)} out of {len(df)} rows.')

    return df_cleaned

In [18]:
train_uncleaned = train.merge(group_ages(age, age_ranges), on='grunnkrets_id', how='left')
train_uncleaned = train_uncleaned.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_uncleaned = train_uncleaned.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_uncleaned = train_uncleaned.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

train_cleaned = clean_out_nan_heavy_rows(train)

Cleaned out 805 out of 12859 rows.


In [23]:
print(len(train_uncleaned), len(train_cleaned))
print(train_uncleaned.isna().sum())
print(train_cleaned.isna().sum())

12859 12054
store_id                                    0
year                                        0
store_name                                  0
plaace_hierarchy_id                         0
sales_channel_name                          0
grunnkrets_id                               0
address                                  1774
lat                                         0
lon                                         0
chain_name                               9122
mall_name                               10579
revenue                                     0
age_0_19                                  805
age_20_39                                 805
age_40_59                                 805
age_60_79                                 805
age_80_90                                 805
grunnkrets_name                            30
district_name                              30
municipality_name                          30
geometry                                   30
area_km2              

In [20]:
train_df = train.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_df = train_df.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_df = train_df.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

assert len(train) == len(train_df)

train = train_df[
    ~(train_df.couple_children_0_to_5_years.isna() | train_df.grunnkrets_name.isna() | train_df.income_all_households.isna())
]





In [21]:
print(len(train), len(train_df))
train.isna().sum()

12732 12859


store_id                                    0
year                                        0
store_name                                  0
plaace_hierarchy_id                         0
sales_channel_name                          0
grunnkrets_id                               0
address                                  1756
lat                                         0
lon                                         0
chain_name                               9030
mall_name                               10479
revenue                                     0
grunnkrets_name                             0
district_name                               0
municipality_name                           0
geometry                                    0
area_km2                                    0
income_all_households                       0
income_singles                              0
income_couple_without_children              0
income_couple_with_children                 0
income_other_households           

In [22]:
train_house_nans = train_house.grunnkrets_id[pd.isnull(train_house.couple_children_0_to_5_years)]
train_spatial_nans = train_spatial.grunnkrets_id[pd.isnull(train_spatial.grunnkrets_name)]
train_income_nans = train_income.income_all_households[pd.isnull(train_income.income_all_households)]

train_cleaned = train[
    ~(train.grunnkrets_id.isin(train_house_nans)) \
    | ~(train.grunnkrets_id.isin(train_spatial_nans)) \
    | ~(train.grunnkrets_id.isin(train_income_nans))
]

print(len(train), len(train_cleaned))
train_cleaned.isna().sum()

train.grunnkrets_id.isin(train_house_nans).value_counts()

NameError: name 'train_house' is not defined

In [ ]:
train_full = train.merge(spatial_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_full = train_full.merge(income_2016.drop(columns=['year']), on='grunnkrets_id', how='left')
train_full = train_full.merge(households_2016.drop(columns=['year']), on='grunnkrets_id', how='left')

train_full.isna().sum()

store_id                                    0
year                                        0
store_name                                  0
plaace_hierarchy_id                         0
sales_channel_name                          0
grunnkrets_id                               0
address                                  1774
lat                                         0
lon                                         0
chain_name                               9122
mall_name                               10579
revenue                                     0
grunnkrets_name                            30
district_name                              30
municipality_name                          30
geometry                                   30
area_km2                                   30
income_all_households                      33
income_singles                             33
income_couple_without_children             33
income_couple_with_children                33
income_other_households           